# Trabalho Final - Introdução à Teoria da Informação

##### Ana Luisa Miranda - 20200017005
##### Helio Lima Correia II - 202000123868

In [10]:
import random
import os

In [11]:
# Função para adicionar uma nova sequência ao dicionário durante a compressão LZW
def adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia):
    proximo_codigo = len(dicionario)  # O próximo código disponível é o tamanho atual do dicionário
    dicionario[nova_sequencia] = proximo_codigo  # Adiciona a nova sequência ao dicionário com o próximo código disponível

In [12]:
# Função para criar o dicionário inicial para o algoritmo LZW
def criar_dicionario_inicial():
    dicionario = {}
    # Adiciona todas as entradas de 0 a 255 (todos os bytes possíveis) ao dicionário inicial
    for i in range(256):
        dicionario[(i.to_bytes(1, byteorder='big', signed=False))] = i
    return dicionario

In [13]:
# Função principal de compressão LZW
def compressao_LZW(nome_arquivo, tamanho_maximo):
    dicionario = criar_dicionario_inicial()  # Cria o dicionário inicial
    tam_max_dic = 2 ** tamanho_maximo  # Define o tamanho máximo do dicionário
    buffer = b''  # Buffer para armazenar a sequência atual
    codigos = []  # Lista para armazenar os códigos gerados pela compressão

    # Abre o arquivo para leitura em modo binário
    with open(nome_arquivo, 'rb') as arquivo:
        while True:
            byte = arquivo.read(1)  # Lê um byte do arquivo

            if not byte:
                break  # Se não houver mais bytes, encerra o loop
            
            nova_sequencia = buffer + byte  # Cria uma nova sequência com o buffer atual e o byte lido

            if nova_sequencia in dicionario:
                buffer = nova_sequencia  # Se a sequência está no dicionário, atualiza o buffer
            else:
                if len(dicionario) < tam_max_dic:  # Verifica se o dicionário não atingiu o tamanho máximo
                    adicionar_nova_sequencia_ao_dicionario(dicionario, nova_sequencia)  # Adiciona a nova sequência ao dicionário
                codigos.append(dicionario[buffer])  # Adiciona o código correspondente ao buffer na lista de códigos
                buffer = byte  # Atualiza o buffer com o byte atual

    if buffer:
        codigos.append(dicionario[buffer])  # Adiciona o código da última sequência no buffer, se houver
    
    return dicionario  # Retorna o dicionário resultante

In [14]:
# Função para reconhecimento de padrões baseada no dicionário gerado
def rec_pad(nome_arquivo, dicionario):
    buffer = b''  # Buffer para armazenar a sequência atual
    codigos = []  # Lista para armazenar os códigos gerados

    # Abre o arquivo para leitura em modo binário
    with open(nome_arquivo, 'rb') as arquivo:
        cabe = arquivo.read(14)  # Lê e descarta o cabeçalho do arquivo (14 bytes)
        while True:
            byte = arquivo.read(1)  # Lê um byte do arquivo

            if not byte:
                break  # Se não houver mais bytes, encerra o loop
            
            nova_sequencia = buffer + byte  # Cria uma nova sequência com o buffer atual e o byte lido

            if nova_sequencia in dicionario:
                buffer = nova_sequencia  # Se a sequência está no dicionário, atualiza o buffer
            else:
                codigos.append(dicionario[buffer])  # Adiciona o código correspondente ao buffer na lista de códigos
                buffer = byte  # Atualiza o buffer com o byte atual

    if buffer:
        codigos.append(dicionario[buffer])  # Adiciona o código da última sequência no buffer, se houver
    
    return len(codigos)  # Retorna o número de códigos gerados

In [15]:
# Função para concatenar vários arquivos em um único arquivo binário
def concatena(arquivos):
    arquivo_saida = 'saida.bin'  # Nome do arquivo de saída

    # Abre o arquivo de saída em modo de escrita binária
    with open(arquivo_saida, 'wb') as arquivo_final:
        # Itera sobre a lista de arquivos
        for nome_arquivo in arquivos:
            # Abre cada arquivo de entrada em modo de leitura binária
            with open(nome_arquivo, 'rb') as arquivo:
                cabe = arquivo.read(14)  # Lê e descarta o cabeçalho do arquivo (14 bytes)
                conteudo = arquivo.read()  # Lê o conteúdo do arquivo
                arquivo_final.write(conteudo)  # Escreve o conteúdo no arquivo de saída
                
    return arquivo_saida  # Retorna o nome do arquivo de saída

In [16]:
# Inicializa os bancos de dados e as amostras
bancos = 40  # Número de bancos
faces = []  # Lista para armazenar os bancos de imagens
amostras = []  # Lista para armazenar as amostras selecionadas

# Preenche a lista de bancos de imagens
for i in range(bancos):
    pessoa = [f'orl_faces/orl_faces/s{i+1}/{j}.pgm' for j in range(1, 11)]  # Cada banco contém 10 imagens
    faces.append(pessoa)

# Seleciona aleatoriamente uma amostra de cada banco
for i in faces:
    indice_aleatorio = random.randint(0, 9)  # Seleciona um índice aleatório entre 0 e 9
    amostras.append(i.pop(indice_aleatorio))  # Remove a imagem selecionada do banco e adiciona à lista de amostras

print(amostras)  # Imprime a lista de amostras
print(f'numero de amostras = {len(amostras)}')  # Imprime o número de amostras selecionadas


['orl_faces/orl_faces/s1/4.pgm', 'orl_faces/orl_faces/s2/4.pgm', 'orl_faces/orl_faces/s3/5.pgm', 'orl_faces/orl_faces/s4/6.pgm', 'orl_faces/orl_faces/s5/1.pgm', 'orl_faces/orl_faces/s6/9.pgm', 'orl_faces/orl_faces/s7/6.pgm', 'orl_faces/orl_faces/s8/1.pgm', 'orl_faces/orl_faces/s9/1.pgm', 'orl_faces/orl_faces/s10/1.pgm', 'orl_faces/orl_faces/s11/2.pgm', 'orl_faces/orl_faces/s12/4.pgm', 'orl_faces/orl_faces/s13/6.pgm', 'orl_faces/orl_faces/s14/7.pgm', 'orl_faces/orl_faces/s15/6.pgm', 'orl_faces/orl_faces/s16/3.pgm', 'orl_faces/orl_faces/s17/10.pgm', 'orl_faces/orl_faces/s18/1.pgm', 'orl_faces/orl_faces/s19/2.pgm', 'orl_faces/orl_faces/s20/8.pgm', 'orl_faces/orl_faces/s21/6.pgm', 'orl_faces/orl_faces/s22/3.pgm', 'orl_faces/orl_faces/s23/10.pgm', 'orl_faces/orl_faces/s24/4.pgm', 'orl_faces/orl_faces/s25/4.pgm', 'orl_faces/orl_faces/s26/2.pgm', 'orl_faces/orl_faces/s27/9.pgm', 'orl_faces/orl_faces/s28/3.pgm', 'orl_faces/orl_faces/s29/1.pgm', 'orl_faces/orl_faces/s30/8.pgm', 'orl_faces/orl_f

In [17]:
# Inicializa variáveis para armazenar os resultados
RCs = []
aux2 = []
aux3 = []

# Loop para testar diferentes valores de K (tamanho máximo do dicionário)
for k in range(9, 17):
    dicionarios = []

    # Concatena as imagens de cada banco e realiza a compressão LZW
    for i in range(bancos):
        aux = concatena(faces[i])
        dicionarios.append(compressao_LZW(aux, k))

    # Calcula a taxa de compressão para cada amostra usando cada dicionário
    for i in amostras:
        for j in dicionarios:
            compressao = rec_pad(i, j)  # Calcula o número de códigos gerados para a amostra
            file_stats = os.stat(i)  # Obtém o tamanho do arquivo original
            RC = compressao / file_stats.st_size  # Calcula a taxa de compressão
            aux2.append(RC)  # Adiciona a taxa de compressão à lista auxiliar
        aux3.append(aux2.copy())  # Adiciona os resultados de compressão para a amostra
        aux2.clear()  # Limpa a lista auxiliar
    RCs.append(aux3.copy())  # Adiciona os resultados para o valor atual de K
    aux3.clear()  # Limpa a lista auxiliar

In [18]:
# Loop para calcular e imprimir as taxas de acerto e erro para cada valor de K
for k in range(8):
    erro = 0
    acertos = 0
    print(f'k = {k+9}')
    for i in range(len(amostras)):
        menor = RCs[k][i][i]  # Assume que o menor valor inicial é o da própria amostra
        for j in range(len(dicionarios)):
            if RCs[k][i][j] < menor:
                menor = RCs[k][i][j]  # Atualiza o menor valor se encontrar uma taxa de compressão menor
        if RCs[k][i][i] == menor:
            acertos += 1  # Conta como acerto se a menor taxa for a da própria amostra
        else:
            erro += 1  # Conta como erro caso contrário
    print(f'acertos = {acertos / len(amostras) * 100}%, erros = {erro / len(amostras) * 100}%')  # Imprime as taxas de acerto e erro


k = 9
acertos = 25.0%, erros = 75.0%
k = 10
acertos = 20.0%, erros = 80.0%
k = 11
acertos = 15.0%, erros = 85.0%
k = 12
acertos = 32.5%, erros = 67.5%
k = 13
acertos = 67.5%, erros = 32.5%
k = 14
acertos = 60.0%, erros = 40.0%
k = 15
acertos = 92.5%, erros = 7.5%
k = 16
acertos = 95.0%, erros = 5.0%
